In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

import intvalpy as ip

# for saving gif
from PIL import Image

# for interactive work
from ipywidgets import interact

In [2]:
# To create a gif image you need to create N pictures, 
# but there is no need to display them. So let's suppress the rendering of graphs
plt.ioff()

# As a model system, consider the example proposed by S.P. Shary
A, b = ip.Shary(4)
n, m = A.shape

# For convenience, let's calculate the boundaries in which each variable changes
consistency = 'uni'
if consistency == 'uni':
    external_evaluation = ip.linear.PSS(A, b)
elif consistency == 'tol':
    external_evaluation = ip.linear.Rohn(A, b, consistency='tol')
else:
    raise Exception('The specified set of solutions is not provided.')

    
# Let's specify which variables we want to fixed
nu = 1
index = np.delete(np.arange(m, dtype=np.int32), nu)

# As a result, we get a "working" matrix
wA = A[:, index].copy

# Using the previously calculated external estimate, indicate the range within which the graph 
# will be plotted and how the fixed variable changes
bounds = [np.array(external_evaluation[index].a, dtype=np.float64), 
          np.array(external_evaluation[index].b, dtype=np.float64)]
inf = np.float64(external_evaluation[nu].a)
sup = np.float64(external_evaluation[nu].b)
step = np.float64(external_evaluation[nu].wid/20)

# we calculate the vertices of the graphs, with the nu-th variable fixed
vertex = []
for x in np.arange(inf, sup, step):
    wb = b - A[:, nu] * x
    vertex.append(ip.IntLinIncR3(wA, wb, consistency=consistency, s=10, bounds=bounds, size=(7,7), show=False))
    

color='C0'
alpha=0.5
s=10
size=(8,8)
frames = []         # A list for storing frames
for vertices in vertex:
    gxmin, gymin, gzmin = ip.infinity, ip.infinity, ip.infinity
    gxmax, gymax, gzmax = -ip.infinity, -ip.infinity, -ip.infinity
    for el in vertices:
        for v in el:
            gxmin = min(gxmin, np.min(v[:, 0]))
            gymin = min(gymin, np.min(v[:, 1]))
            gzmin = min(gzmin, np.min(v[:, 2]))

            gxmax = max(gxmax, np.max(v[:, 0]))
            gymax = max(gymax, np.max(v[:, 1]))
            gzmax = max(gzmax, np.max(v[:, 2]))
    
    l = 0
    fig = plt.figure(figsize=size)
    ax = fig.add_subplot(111, projection='3d')
    
    ax.set_xlim((np.float64(external_evaluation[0].a), np.float64(external_evaluation[0].b)))
    ax.set_ylim((np.float64(external_evaluation[2].a), np.float64(external_evaluation[2].b)))
    ax.set_zlim((np.float64(external_evaluation[3].a), np.float64(external_evaluation[3].b)))
    
    color1 = color
    for el in vertices:

        color = color1
        l = 0
        for v in el:
            x, y, z = v[:, 0], v[:, 1], v[:, 2]
            xmin, ymin, zmin = np.min(abs(x)), np.min(abs(y)), np.min(abs(z))
            xmax, ymax, zmax = np.max(abs(x)), np.max(abs(y)), np.max(abs(z))

            if n <= l and l < n + 3 and (xmin * ymin * zmin == 0):
                l += 1
                continue
                
            elif ((xmax==xmin and xmax==0 and abs(gxmin*gxmax) > 1e-14) or
                  (ymax==ymin and ymax==0 and abs(gymin*gymax) > 1e-14) or
                  (zmax==zmin and zmax==0 and abs(gzmin*gzmax) > 1e-14)):      # demo
                continue

            elif l >= n:
                color = 'red'

            poly3d = [list(zip(x, y, z))]
            PC = Poly3DCollection(poly3d, linewidths=1)
            PC.set_alpha(alpha)
            PC.set_facecolor(color)
            ax.add_collection3d(PC)

            ax.plot(x, y, z, color='black', alpha=1)
            ax.scatter(x, y, z, s=s, color='black')
            
            fig.canvas.draw()
            l += 1
        
    frames.append(np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8'))
    frames[-1] = frames[-1].reshape(fig.canvas.get_width_height()[::-1] + (3,))
    frames[-1] = Image.fromarray(frames[-1])
    
# Let's return the display of drawings
plt.ion()
    
# We take the first frame and add the remaining frames to it.
frames[0].save(
    'Shary4Uni.gif',
    save_all=True,
    append_images=frames[1:],
    optimize=True,
    duration=500,          # Frame display time
    loop=0
)

In [3]:
# Let's turn on the interactivity of the graph
%matplotlib notebook

# we get an interactive slice for the 7-dimensional system
A, b = ip.Shary(7)
n, m = A.shape

# For convenience, let's calculate the boundaries in which each variable changes
consistency = 'tol'
if consistency == 'uni':
    external_evaluation = ip.linear.PSS(A, b)
elif consistency == 'tol':
    external_evaluation = ip.linear.Rohn(A, b, consistency='tol')
else:
    raise Exception('The specified set of solutions is not provided.')
    
# Let's specify which variables we want to fixed
nu = np.array([0, 3, 4, 6])
index = np.delete(np.arange(A.shape[1], dtype=np.int32), nu)

# As a result, we get a "working" matrix
wA = A[:, index].copy

# Using the previously calculated external estimate, indicate the range within which the graph 
# will be plotted and how the fixed variable changes
bounds = [np.array(external_evaluation[index].a, dtype=np.float64), 
          np.array(external_evaluation[index].b, dtype=np.float64)]
inf = np.float64(external_evaluation[nu].a)
sup = np.float64(external_evaluation[nu].b)
step = np.float64(external_evaluation[nu].wid/20)
init = np.float64(external_evaluation[nu].mid)

@interact(x1=(inf[0], sup[0], step[0]), 
          x4=(inf[1], sup[1], step[1]), 
          x5=(inf[2], sup[2], step[2]), 
          x7=(inf[3], sup[3], step[3]))
def update(x1=init, x4=init, x5=init, x7=init):
    # Let us construct an admissible set of solutions. We can switch 
    # to Kaucher arithmetic to obtain the optimal estimation
    wb = b + (A[:, nu] @ np.array([x1, x4, x5, x7])).opp
    vertices = ip.IntLinIncR3(wA, wb, consistency=consistency, s=10, bounds=bounds, size=(7,7))


interactive(children=(FloatSlider(value=1.1102230246251565e-16, description='x1', max=0.8571428571428571, min=…